In [1]:
import pandas as pd
import sklearn
import torch
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
Train_Data = pd.read_csv('train.csv')
Numeric_list = Train_Data.select_dtypes(include=['int', 'float']).columns.tolist()



Numeric_list.remove('Survived')
Numeric_list.remove('Age')

print(Numeric_list)

['PassengerId', 'Pclass', 'SibSp', 'Parch', 'Fare']


In [3]:
from scipy import stats

# Assuming 'numeric_variable' is your numeric variable and 'categorical_variable' is your categorical variable

# Calculate point-biserial correlation coefficient

for i in range(len(Numeric_list)):
    correlation, p_value = stats.pointbiserialr(Train_Data[Numeric_list[i]], Train_Data['Survived'])
    
    print("Point-Biserial Correlation Coefficient between Survival and " + Numeric_list[i] + ":", correlation)
    print("P-value:", p_value)
    print()



Point-Biserial Correlation Coefficient between Survival and PassengerId: -0.005006660767066509
P-value: 0.8813657768798073

Point-Biserial Correlation Coefficient between Survival and Pclass: -0.3384810359610153
P-value: 2.537047387981485e-25

Point-Biserial Correlation Coefficient between Survival and SibSp: -0.03532249888573556
P-value: 0.29224392869829624

Point-Biserial Correlation Coefficient between Survival and Parch: 0.08162940708348347
P-value: 0.014799245374727947

Point-Biserial Correlation Coefficient between Survival and Fare: 0.2573065223849624
P-value: 6.120189341924608e-15



In [4]:
import numpy as np
Categorical_list = Train_Data.select_dtypes(include=['object']).columns.tolist()
Categorical_list.remove('Cabin')

print(Categorical_list)



['Name', 'Sex', 'Ticket', 'Embarked']


In [5]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2

# Assuming 'categorical_var1' and 'categorical_var2' are your categorical variables

# Create a contingency table

for j in range(len(Categorical_list)):
    contingency_table = pd.crosstab(Train_Data[Categorical_list[j]], Train_Data['Survived'])
    
    # Perform the Chi-square test
    chi2_stat, p_val, _, _ = chi2_contingency(contingency_table)
    
    # Determine the degree of association using Cramer's V
    n = contingency_table.sum().sum()
    min_dim = min(contingency_table.shape) - 1
    cramer_v = np.sqrt(chi2_stat / (n * min_dim))
    
    print("Chi-square test statistic:", chi2_stat)
    print("P-value:", p_val)
    print("Cramer's V between Survival and " + Categorical_list[j] + ":", cramer_v)
    print()


Chi-square test statistic: 891.0
P-value: 0.484248151973633
Cramer's V between Survival and Name: 1.0

Chi-square test statistic: 260.71702016732104
P-value: 1.1973570627755645e-58
Cramer's V between Survival and Sex: 0.5409359434469282

Chi-square test statistic: 766.5697029458893
P-value: 0.01152729601163775
Cramer's V between Survival and Ticket: 0.9275492387400215

Chi-square test statistic: 26.48914983923762
P-value: 1.769922284120912e-06
Cramer's V between Survival and Embarked: 0.17261682709984438



In [6]:
from torch import nn

# Make device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [7]:
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Extracted_features = ['Pclass', 'Fare'] #'Name', 'Ticket'],
Response = 'Survived'



X = Train_Data[Extracted_features].values
y = Train_Data[Response].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=40)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

# Define hyperparameters
input_size = len(Extracted_features)
hidden_size1 = 64
hidden_size2 = 32
output_size = 1
learning_rate = 0.01
num_epochs = 1000

# Instantiate the model
model = NeuralNet(input_size, hidden_size1, hidden_size2, output_size)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train_tensor)
    outputs = torch.sigmoid(outputs)
    loss = criterion(outputs, y_train_tensor)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
with torch.no_grad():
    model.eval()
    y_pred = model(X_test_tensor)
    y_pred = torch.sigmoid(y_pred)
    test_loss = criterion(y_pred, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

Epoch [100/1000], Loss: 0.5652
Epoch [200/1000], Loss: 0.5599
Epoch [300/1000], Loss: 0.5571
Epoch [400/1000], Loss: 0.5535
Epoch [500/1000], Loss: 0.5508
Epoch [600/1000], Loss: 0.5482
Epoch [700/1000], Loss: 0.5500
Epoch [800/1000], Loss: 0.5396
Epoch [900/1000], Loss: 0.5378
Epoch [1000/1000], Loss: 0.5361
Test Loss: 0.6271


## model accuracy

In [8]:
# Compute accuracy
y_pred_binary = (y_pred == 1).float()
correct = (y_pred_binary == y_test_tensor).sum().item()
total = y_test_tensor.size(0)
accuracy = correct / total

print(f'Accuracy: {accuracy:.4f}')


Accuracy: 0.5810
